In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt

# Define file paths
file_path = 'chat.csv'  # Updated to use the correct file name
output_file = 'april_2025_chats.csv'

# Read the CSV file with pandas
df = pd.read_csv(file_path, header=None, names=['raw_data'], encoding='utf-8')

# Extract date, time, and message using regex
pattern = re.compile(r'(\d{2}/\d{2}/\d{4}), (\d{2}:\d{2}) - (.+)')

# Function to parse each row
def parse_message(row):
    match = pattern.search(row)
    if match:
        date, time, message = match.groups()
        return pd.Series([date, time, message])
    return pd.Series([None, None, None])

# Apply parsing function
extracted = df['raw_data'].apply(lambda x: parse_message(x) if isinstance(x, str) else pd.Series([None, None, None]))
extracted.columns = ['date', 'time', 'message']

# Combine with original dataframe
df = pd.concat([df, extracted], axis=1)

# Remove rows where date is None (parsing failed)
df = df.dropna(subset=['date'])

# Convert date to datetime for easier filtering
df['datetime'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Display the last 50 entries
print("Last 50 entries:")
print(df.tail(50))

# Filter for April data
april_data = df[df['datetime'].dt.month == 4]

# Save April data to new CSV file
april_data.to_csv(output_file, index=False)

print(f"April data saved to {output_file}")
print(f"Total messages: {len(df)}")
print(f"April messages: {len(april_data)}")

In [ ]:


# Pattern for NET tickets
net_pattern = re.compile(r'NET\d+', re.IGNORECASE)

# Pattern for link down reports from LNOC
link_down_pattern = re.compile(r'link\s+down|outage|connection\s+lost', re.IGNORECASE)
lnoc_pattern = re.compile(r'LNOC', re.IGNORECASE)

# Prepare dataframe
april_data = april_data.copy()  # Avoid SettingWithCopyWarning
april_data['full_datetime'] = pd.to_datetime(april_data['date'] + ' ' + april_data['time'], format='%d/%m/%Y %H:%M')
april_data_sorted = april_data.sort_values('full_datetime')

unique_nets = {}
link_down_reports = []

# First pass: gather link down reports from LNOC
for _, row in april_data_sorted.iterrows():
    msg = row['message']
    if isinstance(msg, str) and lnoc_pattern.search(msg) and link_down_pattern.search(msg):
        link_down_reports.append({
            'message': msg,
            'full_datetime': row['full_datetime']
        })

# Extract NET tickets
for _, row in april_data_sorted.iterrows():
    msg = row['message']
    if isinstance(msg, str):
        tickets = net_pattern.findall(msg)
        for ticket in tickets:
            ticket_key = ticket.upper()
            if ticket_key not in unique_nets:
                # Find the nearest previous link down report
                closest_report = None
                closest_time_diff = None
                for report in link_down_reports:
                    if report['full_datetime'] < row['full_datetime']:
                        time_diff = row['full_datetime'] - report['full_datetime']
                        if closest_time_diff is None or time_diff < closest_time_diff:
                            closest_time_diff = time_diff
                            closest_report = report['message']
                
                unique_nets[ticket_key] = {
                    'ticket': ticket_key,
                    'date': row['date'],
                    'time': row['time'],
                    'link_down_report': closest_report,
                    'time_since_report': str(closest_time_diff) if closest_time_diff else 'No prior report'
                }

# Convert to DataFrame
nets_df = pd.DataFrame.from_dict(unique_nets.values())
nets_df = nets_df.sort_values(['date', 'time'])

if not nets_df.empty:
    print(f"Found {len(nets_df)} unique NET tickets in April 2025.")
    display(nets_df)
    nets_df.to_csv("april_2025_nets.csv", index=False)
    print("Saved to april_2025_nets.csv")
else:
    print("No NET tickets found in April 2025 data.")
